# Get Data

In [18]:
import os
import itertools 
import requests
import rasterio
from rasterio.plot import show
from io import BytesIO

# Define output directory
output_dir = "./data"
os.makedirs(output_dir, exist_ok=True)

# Define variables and coordinates
variables = ["treecover2000", "lossyear"]
coordinates = ["10N_090E", "10N_100E", "00N_090E", "00N_100E", "10N_110E", "00N_110E"]

# Generate URLs for all variables and coordinates
urls = [
    f"https://storage.googleapis.com/earthenginepartners-hansen/GFC-2023-v1.11/Hansen_GFC-2023-v1.11_{var}_{coord}.tif"
    for var, coord in itertools.product(variables, coordinates)
]

# Load raster data and save locally
forest_tiles = []
for url in urls:
    try:
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful
        filename = os.path.join(output_dir, os.path.basename(url))  # Create local filename
        with open(filename, 'wb') as f:
            f.write(response.content)  # Save the file
        # Load the raster data into rasterio
        with rasterio.open(filename) as src:
            print(f"Successfully loaded and saved: {url}")
            forest_tiles.append(src.read(1))  # Reading the first band
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {url}: {e}")


Successfully loaded and saved: https://storage.googleapis.com/earthenginepartners-hansen/GFC-2023-v1.11/Hansen_GFC-2023-v1.11_treecover2000_10N_090E.tif
Successfully loaded and saved: https://storage.googleapis.com/earthenginepartners-hansen/GFC-2023-v1.11/Hansen_GFC-2023-v1.11_treecover2000_10N_100E.tif
Successfully loaded and saved: https://storage.googleapis.com/earthenginepartners-hansen/GFC-2023-v1.11/Hansen_GFC-2023-v1.11_treecover2000_00N_090E.tif
Successfully loaded and saved: https://storage.googleapis.com/earthenginepartners-hansen/GFC-2023-v1.11/Hansen_GFC-2023-v1.11_treecover2000_00N_100E.tif
Successfully loaded and saved: https://storage.googleapis.com/earthenginepartners-hansen/GFC-2023-v1.11/Hansen_GFC-2023-v1.11_treecover2000_10N_110E.tif
Successfully loaded and saved: https://storage.googleapis.com/earthenginepartners-hansen/GFC-2023-v1.11/Hansen_GFC-2023-v1.11_treecover2000_00N_110E.tif
Successfully loaded and saved: https://storage.googleapis.com/earthenginepartners-

# Resampling

In [12]:
output_workspace = r"C:\Users\godfr\Documents\ArcGIS\Projects\Deforestation\Deforestation.gdb"

In [19]:
import arcpy
import os

# Set workspace directory where your raster files are located
workspace = r"C:\Users\godfr\Documents\ArcGIS\Projects\Deforestation\data"
arcpy.env.workspace = workspace

# Output geodatabase or folder for resampled rasters
output_workspace = r"C:\Users\godfr\Documents\ArcGIS\Projects\Deforestation\Deforestation.gdb"

# Define cell size and resampling type
cell_size = "0.005 0.005"
resampling_type = "NEAREST"

# Loop through each .tif file in the workspace
for raster in arcpy.ListRasters("*.tif"):
    # Define input raster and sanitized output raster name
    in_raster = os.path.join(workspace, raster)
    sanitized_name = raster[22:-4] + "_Resample"  # Extract from character 23 to the end and add "_Resample"
    out_raster = os.path.join(output_workspace, sanitized_name)
    
    # Perform resampling
    print(f"Resampling {raster} to {sanitized_name}...")
    try:
        arcpy.management.Resample(in_raster, out_raster, cell_size, resampling_type)
    except arcpy.ExecuteError as e:
        print(f"Error resampling {raster}: {e}")

print("Resampling completed!")

Resampling Hansen_GFC-2023-v1.11_lossyear_00N_090E.tif to lossyear_00N_090E_Resample...
Resampling Hansen_GFC-2023-v1.11_lossyear_00N_100E.tif to lossyear_00N_100E_Resample...
Resampling Hansen_GFC-2023-v1.11_lossyear_00N_110E.tif to lossyear_00N_110E_Resample...
Resampling Hansen_GFC-2023-v1.11_lossyear_10N_090E.tif to lossyear_10N_090E_Resample...
Resampling Hansen_GFC-2023-v1.11_lossyear_10N_100E.tif to lossyear_10N_100E_Resample...
Resampling Hansen_GFC-2023-v1.11_lossyear_10N_110E.tif to lossyear_10N_110E_Resample...
Resampling Hansen_GFC-2023-v1.11_treecover2000_00N_090E.tif to treecover2000_00N_090E_Resample...
Resampling Hansen_GFC-2023-v1.11_treecover2000_00N_100E.tif to treecover2000_00N_100E_Resample...
Resampling Hansen_GFC-2023-v1.11_treecover2000_00N_110E.tif to treecover2000_00N_110E_Resample...
Resampling Hansen_GFC-2023-v1.11_treecover2000_10N_090E.tif to treecover2000_10N_090E_Resample...
Resampling Hansen_GFC-2023-v1.11_treecover2000_10N_100E.tif to treecover2000_10N

# Mosaicing

In [7]:
import arcpy

# Set environment
arcpy.env.workspace = r"C:\Users\godfr\Documents\ArcGIS\Projects\Deforestation"
output_location = r"C:\Users\godfr\Documents\ArcGIS\Projects\Deforestation\Deforestation.gdb"

# Define raster lists
treecover2000_rasters = [
    "treecover2000_10N_090E_Resample",
    "treecover2000_10N_100E_Resample",
    "treecover2000_10N_110E_Resample",
    "treecover2000_00N_090E_Resample",
    "treecover2000_00N_100E_Resample",
    "treecover2000_00N_110E_Resample"
]

lossyear_rasters = [
    "lossyear_10N_090E_Resample",
    "lossyear_10N_100E_Resample",
    "lossyear_10N_110E_Resample",
    "lossyear_00N_090E_Resample",
    "lossyear_00N_100E_Resample",
    "lossyear_00N_110E_Resample"
]

# Ensure no existing outputs
output_raster_treecover = r"C:\Users\godfr\Documents\ArcGIS\Projects\Deforestation\Deforestation.gdb\treecover2000_merged"
output_raster_lossyear = r"C:\Users\godfr\Documents\ArcGIS\Projects\Deforestation\Deforestation.gdb\lossyear_merged"

if arcpy.Exists(output_raster_treecover):
    arcpy.management.Delete(output_raster_treecover)
if arcpy.Exists(output_raster_lossyear):
    arcpy.management.Delete(output_raster_lossyear)

# Check and define spatial reference
spatial_ref = arcpy.SpatialReference(4326)
for raster in treecover2000_rasters + lossyear_rasters:
    desc = arcpy.Describe(raster)
    if not hasattr(desc, "spatialReference") or desc.spatialReference is None:
        print(f"Defining spatial reference for raster: {raster}")
        arcpy.management.DefineProjection(raster, spatial_ref)

# Mosaic treecover2000
arcpy.management.MosaicToNewRaster(
    input_rasters=";".join(treecover2000_rasters),
    output_location=output_location,
    raster_dataset_name_with_extension="treecover2000_merged",
    coordinate_system_for_the_raster=spatial_ref,
    pixel_type="8_BIT_UNSIGNED",
    cellsize=None,
    number_of_bands=1,
    mosaic_method="LAST",
    mosaic_colormap_mode="FIRST"
)

# Mosaic lossyear
arcpy.management.MosaicToNewRaster(
    input_rasters=";".join(lossyear_rasters),
    output_location=output_location,
    raster_dataset_name_with_extension="lossyear_merged",
    coordinate_system_for_the_raster=spatial_ref,
    pixel_type="8_BIT_UNSIGNED",
    cellsize=None,
    number_of_bands=1,
    mosaic_method="LAST",
    mosaic_colormap_mode="FIRST"
)

print("Mosaicing completed successfully!")

Mosaicing completed successfully!


# Merge Sumatra with Borneo

In [8]:
# Input datasets (relative to the workspace)
input_datasets = ["Sumatra", "borneo"]

# Output feature class (within the geodatabase)
output_path = r"Deforestation.gdb\Sumatra_Borneo"

# Define field mappings
field_mappings = arcpy.FieldMappings()

# Automatically create field mappings for all inputs
for dataset in input_datasets:
    field_mappings.addTable(dataset)

# Perform the merge
arcpy.management.Merge(
    inputs=input_datasets,
    output=output_path,
    field_mappings=field_mappings,
    add_source="NO_SOURCE_INFO"
)

print(f"Merging Sumatra and Borneo completed successfully! Output saved to: {output_path}")

Merging Sumatra and Borneo completed successfully! Output saved to: Deforestation.gdb\Sumatra_Borneo


# Clip Rasters

In [13]:
from arcpy.sa import ExtractByMask

with arcpy.EnvManager(scratchWorkspace=r"C:\Users\godfr\Documents\ArcGIS\Projects\Deforestation\Deforestation.gdb"):
    # Define inputs and outputs
    rasters = {
        "lossyear_merged": "lossyear_masked",
        "treecover2000_merged": "treecover2000_masked"
    }
    mask_data = "Sumatra_Borneo"
    analysis_extent = '95.1935543765559 -5.94328835184967 119.269555210742 7.03754734990478 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'

    for in_raster, out_raster_name in rasters.items():
        # Extract by mask
        out_raster = ExtractByMask(
            in_raster=in_raster,
            in_mask_data=mask_data,
            extraction_area="INSIDE",
            analysis_extent=analysis_extent
        )
        # Save the result
        out_raster.save(f"{output_workspace}/{out_raster_name}")

print("Mask extraction completed for lossyear_merged and treecover2000_merged!")


Mask extraction completed for lossyear_merged and treecover2000_merged!


'lossyear_masked'

In [16]:
import arcpy
from arcpy.sa import Con

# Extract only forest loss pixels (1-20) and set the rest as NoData
loss_pixels = Con(
    in_conditional_raster="lossyear_masked",
    in_true_raster_or_constant=1,  # Value for forest loss
    in_false_raster_or_constant=None,  # NoData for other values
    where_clause="Value >= 1 AND Value <= 20"
)

# Save the new raster layer with only forest loss pixels
loss_pixels.save(fr"{output_workspace}/lossyear_loss_only")

# Apply symbology: Red for forest loss
color_map = [
    [1, 255, 0, 0]  # Value 1 -> Red
]

# Add a color map to the raster
arcpy.management.AddColormap(
    in_raster=f"{output_workspace}/lossyear_loss_only",
    input_colormap=color_map
)

print("Forest loss layer created and symbolized with red.")


TypeError: AddColormap() got an unexpected keyword argument 'input_colormap'

In [17]:
import arcpy

# Define a new color map: 1 (loss) -> Red, NoData -> Transparent
color_map = [
    [1, 255, 0, 0],  # Value 1 (forest loss) -> Red
]

# Apply the color map
arcpy.management.AddColormap(
    in_raster="Deforestation.gdb/lossyear_symbology",
    input_colormap=color_map
)

print("Color map applied: Red for forest loss, transparent for no loss!")


TypeError: AddColormap() got an unexpected keyword argument 'input_colormap'